In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
cd '../gdrive/MyDrive/SSAC/3조'

/gdrive/.shortcut-targets-by-id/15_BxZVEQYCdGCGiQ5nexpWPc1cgHVe4w/3조


In [3]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import requests
import os
import urllib.request

import time

import cv2
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import transforms

# DeepLab

## DeepLab

In [4]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_mobilenet_v3_large', pretrained=True)
model.eval()

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth


  0%|          | 0.00/161M [00:00<?, ?B/s]

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [5]:
def deeplab(image):
    input_image = Image.fromarray(image)
    input_image = input_image.convert("RGB")

    preprocess = transforms.Compose([
        # transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)['out'][0]
    output_predictions = output.argmax(0)

    # tensor to numpy array
    try: 
        mask = output_predictions.cpu().numpy()
    except:
        mask = output_predictions.numpy()

    # 1보다 크면 1, 0이면 0
    mask = np.where(mask>0, 1, 0)
    return mask

## 시각화 함수

In [6]:
def show_img_compar5(img_1, img_2, img_3, img_4, img_5):
    f, ax = plt.subplots(1, 5, figsize=(10,10))
    ax[0].imshow(img_1)
    ax[1].imshow(img_2)
    ax[2].imshow(img_3)
    ax[3].imshow(img_4)
    ax[4].imshow(img_5)
    ax[0].axis('off') #hide the axis
    ax[1].axis('off')
    ax[2].axis('off')
    ax[3].axis('off')
    ax[4].axis('off')
    f.tight_layout()
    plt.show()

## DeepLab으로 배경/상품 색 추출

In [7]:
# RGB 색상표
color_dict = {
    # 피부색 계열
    'Bisque' : [255, 228, 196],
    'Blanchedalmond': [255, 235, 205],
    'Ivory': [255, 255, 240],
    # 무채색 계열
    'Black': [0,0,0], 
    'Gray': [128, 128, 128],
    'White': [255, 255, 255],
    'Silver': [192, 192, 192],
    # 유채색 계열
    'Red': [255, 0, 0],
    'Maroon': [128, 0, 0],
    'Orange': [255, 165, 0],
    'Gold': [255, 215, 0],
    'Olive': [128, 128, 0],
    'Yellow': [255, 255, 0],
    'Chartreuse': [127, 255, 0],
    'Green': [0, 128, 0],
    'Lime': [0, 255, 0],
    'Teal': [0, 128, 128],
    'Aqua': [0, 255, 255],
    'Skyblue': [135, 206, 235],
    'Navy': [0, 0, 128],
    'Blue': [0, 0, 255],
    'Blueviolet': [138, 43, 226],
    'Purple': [128, 0, 128],
    'Magenta': [255, 0, 255],
    'Pink': [255, 192, 203],
    # 갈색 계열
    'Wheat': [245, 222, 179],
    'Saddlebrown': [139, 69, 19],
    'Tan': [210, 180, 140],
    'Linen': [250, 240, 230]
    }

def rgb_to_color(rgb, color_dict=color_dict):
    color_distance = [np.linalg.norm(np.array(rgb)-col) for col in color_dict.values()]
    return list(color_dict.keys())[np.argmin(color_distance)]

## Entropy

In [8]:
from skimage.io import imread, imshow
from skimage.util import img_as_ubyte
from skimage.filters.rank import entropy
from skimage.morphology import disk

def get_entropy(image_gray):
    scaled_entropy = image_gray / image_gray.max()
    entropy_image = entropy(img_as_ubyte(scaled_entropy), disk(6))
    scaled_entropy = entropy_image / entropy_image.max()
    mask_ent = scaled_entropy > 0.8
    return mask_ent

In [9]:
def get_image_feature(cid):
    url = f'https://media.bunjang.co.kr/product/{cid}_...'
    try: 
        image_nparray = np.asarray(bytearray(requests.get(url).content), dtype=np.uint8)
        image = cv2.imdecode(image_nparray, cv2.IMREAD_COLOR)
        # image = cv2.resize(image, dsize=(224, 224), interpolation=cv2.INTER_AREA)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        mask = deeplab(image_rgb)

        image_nobg = image_rgb * mask[:, :, np.newaxis]
        image_bg = image_rgb * (1-mask)[:, :, np.newaxis]

        background_ratio = ((1-mask).sum())/(image_rgb.shape[0]*image_rgb.shape[1])

        bg_color = [np.delete(image_rgb[:,:,i], (mask.reshape(-1)==1)) for i in range(3)]
        nobg_color = [np.delete(image_rgb[:,:,i], (mask.reshape(-1)==0)) for i in range(3)]

        # 배경/상품 색상 추출    
        rgb_bg = np.array([int(np.median(temp)) for temp in bg_color])
        color_bg = rgb_to_color(rgb_bg)

        rgb_nobg = np.array([int(np.median(temp)) for temp in nobg_color])
        color_nobg = rgb_to_color(rgb_nobg)


        # 노이즈 계산
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        bg_gray = image_gray * (1-mask)
        bg_ent = get_entropy(bg_gray)
        bg_noise = bg_ent.sum() / (1-mask).sum()

        nobg_gray = image_gray * mask
        nobg_ent = get_entropy(nobg_gray)
        nobg_noise = nobg_ent.sum() / mask.sum()

        # HSV -> 사람 구분
        upper_human = (20, 150, 200)
        lower_human = (0, 50, 60)
        image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        human_mask = cv2.inRange(image_hsv, lower_human, upper_human)
        _,counts = np.unique(mask * human_mask, return_counts=True)
        human_ratio = counts[1] / mask.sum()

        image_dict = {"pid": cid, 'background_ratio': background_ratio,
        'bg_rgb': rgb_bg, "nobg_rgb": rgb_nobg, 
        "bg_color": color_bg, "nobg_color": color_nobg, 
        "bg_noise": bg_noise, "nobg_noise": nobg_noise,
        "human_ratio": human_ratio}

    except:
        image_dict = {"pid": cid, 'background_ratio': -1,
        'bg_rgb': -1, "nobg_rgb": -1, 
        "bg_color": -1, "nobg_color": -1, 
        "bg_noise": -1, "nobg_noise": -1,
        "human_ratio": -1}

    return image_dict

# 크롤링

In [10]:
cat_list = [320040]

num_contents = 500
num_pages = num_contents//100 + 1

total_info = []
for cat_id in cat_list:
    infos = []
    for page_num in tqdm(range(1,num_pages)):
        url = f"https://api.bunjang.co.kr/api/1/find_v2.json?f_category_id={cat_id}&page={page_num}&order=date&req_ref=category&stat_device=w&n=100&version=4"
        response = requests.get(url)
        ids = [item['pid'] for item in response.json()['list']]
        for pid in tqdm(ids):
            try: 
                url = f'https://api.bunjang.co.kr/api/1/product/{pid}/detail_info.json?version=4'
                res = requests.get(url)
                item_info = res.json()['item_info']
                # item_info.pop('description')
                item_info.pop('description_for_detail')
                seller_info = res.json()['seller_info']
                image_info = get_image_feature(pid)
                info = {**item_info, **seller_info, **image_info}
                
                infos.append(info)
            except:
                pass
        # 한페이지 끝날때마다 저장
        # new_crawling_중분류.csv
        # 덮어쓰기
        pd.DataFrame(infos).to_csv(f'new_crawling_{cat_id}.csv', index=False)
        # 페이지별로 따로 저장
        # pd.DataFrame(infos).to_csv(f'new_crawling_{cat_id}_{page_num}.csv', index=False)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]